# Sequence to Sequence ამოცანები

Sequence to Sequence ამოცანა შეიძლება განვსაზღვროთ როგორც ფუნქცია f, რომელიც გარდაქმნის შემავალ მიმდევრობას X შესაბამის გამომავალ მიმდევრობაში Y :

f: X → Y
სადაც:
- X = (x₁, x₂, ..., xₙ) არის შემავალი მიმდევრობა (Input Sequence)
- Y = (y₁, y₂, ..., yₘ) არის გამომავალი მიმდევრობა (Output Sequence)

ყურადღება მიაქციეთ, რომ n და m შეიძლება იყოს განსხვავებული სიგრძის, ანუ შემავალი და გამომავალი მიმდევრობების სიგრძეები შეიძლება არ იყოს ერთმანეთის ტოლი.

seq-to-seq ამოცანები შეიძლება შეეხებოდეს, როგორც ტექსტის მიმდევრობებს, ასევე ნებისმიერ სხვა მოდალობას, როგორებიცაა აუდიო სიგნალები, ფოტო, ვიდეო და სხვა. ამ ტიპის ამოცანების მაგალითებია:

- მანქანური თარგმანი:
    - შემავალი მიმდევრობა: წინადადება ერთ ენაზე
    - გამომავალი მიმდევრობა: იგივე წინადადება სხვა ენაზე
    - მაგალითი: "Hello, how are you?" → "გამარჯობა, როგორ ხარ?"

- ტექსტის შეჯამება:
    - შემავალი მიმდევრობა: გრძელი ტექსტი ან დოკუმენტი
    - გამომავალი მიმდევრობა: ტექსტის მოკლე შეჯამება
    - მაგალითი: [გრძელი სტატია] → "სტატია აღწერს კლიმატის ცვლილების გავლენას ოკეანეებზე"

- დიალოგის სისტემები:
    - შემავალი მიმდევრობა: მომხმარებლის შეკითხვა ან მოთხოვნა
    - გამომავალი მიმდევრობა: სისტემის პასუხი
    - მაგალითი: "რა არის დღეს ამინდის პროგნოზი?" → "დღეს მოსალოდნელია მზიანი ამინდი, მაქსიმალური ტემპერატურა 25°C"

- კოდის გენერაცია:
    - შემავალი მიმდევრობა: პროგრამის აღწერა ბუნებრივ ენაზე
    - გამომავალი: პროგრამული კოდი
    - მაგალითი: "შექმენი ფუნქცია, რომელიც დაითვლის ფიბონაჩის მიმდევრობას" → [შესაბამისი Python კოდი]

- ტექსტიდან მეტყველების სინთეზი:
    - შემავალი მიმდევრობა: წერილობითი ტექსტი
    - გამომავალი მიმდევრობა: ფონემების ან აუდიო სიგნალების მიმდევრობა
    - მაგალითი: "გამარჯობა" → [სიტყვის შესაბამისი გახმოვანება]

- ხმის ამოცნობა (ტრანსკრიფცია):
    - შემავალი მიმდევრობა: აუდიო სიგნალების მიმდევრობა
    - გამომავალი მიმდევრობა: ტრანსკრიპტი ტექსტის სახით
    - მაგალითი: [აუდიო ფაილი] → "გამარჯობა, როგორ ხარ?"

<video src="https://lena-voita.github.io/resources/lectures/seq2seq/general/enc_dec_prob_idea.mp4" controls autoplay loop style="display: block; margin: 0 auto" width=600/>

ამგვარი ამოცანების გადაჭრისთვის Encoder-Decoder არქიტექტურა გამოიყენება:

- Encoder - კითხულობს მთელს შემავალი მიმდევრობას X = (x₁, x₂, ..., xₙ) და ქმნის მის კარგ რეპრეზენტაციას;
- Decoder - იყენებს Encoder-ის რეპრეზენტაციას სამიზნე მიმდევრობის დასაგენერირებლად.

როგორც წესი, დეკოდირება ნაბიჯ-ნაბიჯ ხდება. მაგალითად, მანქანური თარგმნის შემთხვევაში, წინადადების თარგმანი სიტყვა-სიტყვა გენერირდება. Decoder თითოეული სიტყვის დაგენერირებამდე გვაძლევს სავარაუდო სიტყვების ალბათურ განაწილებას, საიდანაც შეგვიძლია ამოვირჩიოთ ყველაზე შესაფერისი კანდიდატი.

ეს პროცესი შეიძლება აღვწეროთ შემდეგნაირად:

1. Encoder ამუშავებს შემავალ მიმდევრობას _X_ და ქმნის კონტექსტუალურ რეპრეზენტაციას _C_.
1. Decoder იწყებს გამომავალი მიმდევრობის _Y_ გენერირებას, იყენებს რა _C_-ს და უკვე დაგენერირებულ სიმბოლოებს.
1. თითოეული ნაბიჯისთვის _t_, Decoder აგენერირებს ალბათურ განაწილებას _P(yₜ|y₁, ..., yₜ₋₁, C)_ მომდევნო სიმბოლოსთვის.
1. ამ განაწილებიდან ირჩევა ყველაზე სავარაუდო სიტყვა (ტოკენი).
1. პროცესი გრძელდება, სანამ არ დაგენერირდება სპეციალური დასრულების სიმბოლო ან არ მიიღწევა წინასწარ განსაზღვრული მაქსიმალური სიგრძე.

# Transformer: Attention is All You Need

<img src="https://lena-voita.github.io/resources/lectures/seq2seq/transformer/model-min.png" width=600 style="display: block; margin: 0 auto">

2017 წელს Google-ის მკვლევარებმა თავიანთ ნაშრომში (Attention is All You Need) გამოაქვეყნეს Transformer არქიტექტურა. როგორც სახელიდან ჩანს ამ არქიტექტურის ფუნდამენტს წარმოადგენს Attention მექანიზმი, რომელიც 2014 წლიდან გამოიყენებოდა seq-to-seq ამოცანების გადაჭრაში. მთავარი უპირატესობები, რაც ამ არქიტექტურამ წინამორბედებთან შედარებით აჩვენა:
- შეუძლია შემავალი მიმდევრობის პარალელური დამუშავება GPU-ს გამოყენებით, მაშინ როცა რეკურენტული ნეირონული ქსელები (RNN) შემავალ ინფორმაციას სიტყვა-სიტყვა ამუშავებენ;
- შეუძლია გაცილებით გრძელ ტექსტებთან მუშაობა ხარისხის დაკარგვის გარეშე;
- პარალელურობის გამო მარტივად შეიძლება ამ არქიტექტურის მასშტაბირება;
- გაცილების ჩქარი ტრეინინგი;
- მნიშვნელოვნად გაუმჯობესებული შედეგები ენის მოდელირების, თარგმნის, კითხვებზე პასუხის გაცემის და სხვა NLP ამოცანებში.

<img src="https://lena-voita.github.io/resources/lectures/seq2seq/transformer/rnn_vs_transformer_river-min.png" width=600 style="display: block; margin: 0 auto">

### Encoder

<img src="./images/Encoder.png" width=200 style="display: block; margin: 0 auto">

მთავარი დიაგრამის მარცხენა ნაწილი - Encoder ამუშავებს შემომავალ მიმდევრობას, შედგება N ცალი მიმდევრობით გადაბმული Encoder ბლოკისაგან (ნაცრისფერი ნაწილი), რომელთაგან თითოეული:

1. Self-Attention მექანიზმით:
    - თითოეული ტოკენის რეპრეზენტაცია "ყურადღებას" აქცევს სხვა ტოკენებს.
    - ეს საშუალებას აძლევს მოდელს, გაითვალისწინოს კონტექსტი მთელი მიმდევრობიდან.

1. Feed-Forward ქსელით:
    - ანახლებს/გარდაქმნის კონკრეტული ტოკენის რეპრეზენტაციას.
    - ზრდის მოდელის უნარს, დაამუშაოს რთული პატერნები.

### Decoder

<img src="./images/Decoder.png" width=200 style="display: block; margin: 0 auto">


მთავარი დიაგრამის მარჯვენა ნაწილი - Decoder იღებს Encoder-ის საბოლოო layer-ის რეპრეზენტაციებს და აგენერირებს სამიზნე მიმდევრობას. ისიც, Encoder-ის მსგავსად, შედგება N ცალი მიმდევრობით გადაბმული Decoder ბლოკისაგან (ნაცრისფერი ნაწილი), რომელთაგან თითოეული:

1. Masked Self-Attention მექანიზმით:
    - ამუშავებს უკვე დაგენერირებულ ტოკენებს და "ყურადღებას" აქცევს მათ.

1. Encoder-Decoder Attention მექანიზმით:
    - აკავშირებს Decoder-ის მიმდინარე მდგომარეობას Encoder-ით მიღებულ ტოკენის რეპრეზენტაციებთან.
    - საშუალებას აძლევს Decoder-ს, ფოკუსირდეს შემავალი მიმდევრობის რელევანტურ ნაწილებზე.

1. Feed-Forward ქსელით:
    - ანახლებს/გარდაქმნის თითოეული ტოკენის რეპრეზენტაციას.
    - ზრდის მოდელის უნარს, დაამუშაოს რთული პატერნები.

### ტექსტის გენერაცია

<img src="https://lena-voita.github.io/resources/lectures/seq2seq/transformer/transformer_original.gif" width=600 style="display: block; margin: 0 auto">

ამ ანიმაციაზე ნაჩვენებია ენკოდირებისა და დეკოდირების დროს რეპრეზენტაციების მიმოცვლა.

Encoding:
1. ენკოდირება იწყება სიტყვების "I" "arrived" "at" "the" საწყისი ემბედინგებით (ცარიელი შავი რგოლები);
1. self-attention-ით ხდება ამ ტოკენების რეპრეზენტაციების "გამდიდრება" (ერთმანეთზე ყურადღების მიქცევა);
1. რგოლის შევსება/გაფერადება ნიშნავს feed-forward layer-ის გავლას;
1. self-attention და feed-forward მეორდება 3-ჯერ (ამ ანიმაციაში N=3);

Decoding:
1. ენკოდირებისგან განსხვავებით დეკოდირება არა პარალელურად, არამედ სიტყვა-სიტყვა მიმდინარეობს;
1. აქაც ვიწყებთ საწყისი ემბედინგებით (ცარიელი შავი რგოლები);
1. self-attention-ით უკვე დაგენერირებულ ტოკენებს ექცევა ყურადღება;
1. cross-attention-ით ყურადღება ექცევა encoder-დან წამოსულ ტოკენებს;
1. რგოლის შევსება/გაფერადება ნიშნავს feed-forward layer-ის გავლას;
1. self-attention, cross-attention და feed-forward მეორდება 3-ჯერ (ამ ანიმაციაშიც N=3).

როგორც ხედავთ, Encoder-ისა და Decoder-ის არქიტექტურა თითქმის იდენტურია, განსხვავება მხოლოდ Decoder-ის cross-attention-ია, როდესაც იგი ყურადღებას აქცევს Encoder-ის მიერ შექმნილ შემავალი მიმდევრობის რეპრეზენტაციებს.